In [1]:
import tensorflow as tf
import numpy as np

2023-11-16 08:44:54.445378: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 08:44:54.445514: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 08:44:54.449596: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 08:44:54.785073: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
# Load example MNIST data and pre-process it
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.reshape(-1, 784).astype("float32") / 255.0
X_test = X_test.reshape(-1, 784).astype("float32") / 255.0

# Limit the data to 1000 samples
x_train = X_train[:512]
y_train = Y_train[:512]
x_valid = X_train[512:640]
y_valid = Y_train[512:640]
x_test = X_test[:128]
y_test = Y_test[:128]

In [16]:
def get_model():
    model = tf.keras.Sequential()
    model.add(layer=tf.keras.layers.Dense(units=1, input_dim=784))
    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.1),
        loss="mean_squared_error",
        metrics=["mean_absolute_error"],
    )
    return model

In [4]:
class CustomCallbackh2(tf.keras.callbacks.Callback):
    def __init__(self, *, retio):
        super().__init__()
        # self.model.min_delta = 0
        self.retio = retio
        self.model.min_delta = retio
        self.val_epoch = []

    def on_epoch_end(self, epoch, logs={}):
        self.model.min_delta = logs["val_loss"] * self.retio
        print(self.model.min_delta)

        print(
            "\033[1m"
            + "\033[31m"
            + "End epoch {} of training; got log keys: {}".format(epoch + 1, logs)
            + "\033[0m"
            + "\033[22m"
            + "\033[24m",
        )

In [17]:
class CustomCallbackh(tf.keras.callbacks.Callback):
    def __init__(self, *, retio, patience=0):
        super().__init__()
        self.retio = retio
        self.patience = patience
        self.epoch_val = []
        self.Retio = []

    def on_epoch_end(self, epoch, logs={}):
        self.epoch_val.append(logs["val_loss"])
        self.Retio.append(
            abs(
                (self.epoch_val[epoch] - self.epoch_val[epoch - 1])
                / self.epoch_val[epoch]
            )
        )
        print(
            "\033[1m"
            + "\033[35m"
            + "{}".format(self.Retio[epoch])
            + "\033[0m"
            + "\033[22m"
            + "\033[24m",
        )
        if len(self.Retio) > self.patience:
            if all(x < self.retio for x in self.Retio[-self.patience :]):
                self.model.stop_training = True
                print(
                    "\033[4m"
                    + "\033[1m"
                    + "\033[32m"
                    + "Early stopping"
                    + "\033[0m"
                    + "\033[22m"
                    + "\033[24m",
                )

        print(
            "\033[1m"
            + "\033[31m"
            + "End epoch {} of training; got log keys: {}".format(epoch + 1, logs)
            + "\033[0m"
            + "\033[22m"
            + "\033[24m",
        )
        return self.Retio

In [11]:
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print(
            "Up to batch {}, the average loss is {:7.2f}.".format(
                batch, logs["loss"])
        )

    def on_test_batch_end(self, batch, logs=None):
        print(
            "Test Up to batch {}, the average loss is {:7.2f}.".format(
                batch, logs["loss"]
            )
        )

    def on_epoch_end(self, epoch, logs=None):
        print(
            "The average loss for epoch {} is {:7.2f} "
            "and mean absolute error is {:7.2f}.".format(
                epoch, logs["loss"], logs["mean_absolute_error"]
            )
        )

In [9]:
class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.

    Arguments:
        patience: Number of epochs to wait after min has been hit. After this
        number of no improvement, training stops.
    """

    def __init__(self, patience=0):
        super().__init__()
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None

    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))

In [5]:
np.random.seed(seed=42)
tf.random.set_seed(seed=42)

In [ ]:
model = get_model()
hst = model.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=10,
    verbose=0,
    validation_data=(x_valid, y_valid),
    callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()],
)

In [21]:
model = get_model()
cch = CustomCallbackh(retio=0.8, patience=10)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=hhh)
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=100,
    verbose=0,
    validation_data=(x_valid, y_valid),
    callbacks=[cch, tensorboard_callback],
)

# res = model.evaluate(x=x_test, y=y_test, verbose=0,callbacks=[CustomCallback()])

# res = model.predict(x=x_test, verbose=0, callbacks=[CustomCallback()])

0.0
End epoch 1 of training; got log keys: {'loss': 62.02981185913086, 'mean_absolute_error': 4.334650993347168, 'val_loss': 29.41011619567871, 'val_mean_absolute_error': 4.88128662109375}
0.3047733360410105
End epoch 2 of training; got log keys: {'loss': 28.391149520874023, 'mean_absolute_error': 4.648954391479492, 'val_loss': 42.30291748046875, 'val_mean_absolute_error': 5.915416717529297}
0.10315631676617079
End epoch 3 of training; got log keys: {'loss': 28.44422721862793, 'mean_absolute_error': 4.424105644226074, 'val_loss': 47.168663024902344, 'val_mean_absolute_error': 6.295935153961182}
3.4400778926889246
End epoch 4 of training; got log keys: {'loss': 28.684097290039062, 'mean_absolute_error': 4.276693344116211, 'val_loss': 10.62338638305664, 'val_mean_absolute_error': 2.7517812252044678}
0.6523584973376284
End epoch 5 of training; got log keys: {'loss': 28.81157684326172, 'mean_absolute_error': 4.140886306762695, 'val_loss': 30.558452606201172, 'val_mean_absolute_error': 4.97

In [22]:
%load_ext tensorboard
%tensorboard --logdir=/home/hamze/personal/ML/mylogs --load_fast=false 

Launching TensorBoard...

In [119]:
print(
    "\033[4m"
    + "\033[1m"
    + "\033[32m"
    + "{}".format(5)
    + "\033[0m"
    + "\033[22m"
    + "\033[24m",
    end="\n",
)
print(5)

5
5


In [7]:
import pathlib
import time


def set_path_dir(root_path: str = "mylogs"):
    return pathlib.Path(root_path) / time.strftime(format="run-%Y-%m-%d-%H-%M")

In [20]:
hhh = set_path_dir()

In [14]:
type(hhh)

pathlib.PosixPath